In [ ]:
# imports

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras.preprocessing.image as kim
from keras.backend import tf as ktf
import os
from PIL import Image
from matplotlib import pyplot as plt
import copy

#helpers

sigLev = 3
pd.options.display.precision = sigLev
figWidth = figHeight = 9
inputDir = "../input"


# Motivation

Based on my [previous analysis](https://www.kaggle.com/mmrosenb/whales-an-exploration), it's apparent that we have a severe imbalanced classes problem. Thus, I think we need to spend some time applying transformations to observations in the smaller classes in order to create a more balanced dataset for fitting.

# Find small class

Let's find an extremely small class to work with, perhaps one with just one image.

In [ ]:
trainFrame = pd.read_csv(f'{inputDir}/train.csv')

In [ ]:
classCountFrame = trainFrame.groupby("Id",as_index = False)["Image"].count()
classCountFrame = classCountFrame.rename(columns = {"Image":"count"})
#then order
classCountFrame = classCountFrame.sort_values("count",ascending = True)
#then just check the head
classCountFrame.head()

We see that `w_ffdab7a` seems to be a relatively small class.Turns out from later analysis that this is a greyscaled image. Given that we are looking to augment our dataset with further greyscaling, I find this to not be an optimal choice for an example class. Thus, let's use this class `w_6384242` for our example transformations.

In [ ]:
chosenClass = classCountFrame["Id"].iloc[1]
consideredImageObs = trainFrame[trainFrame["Id"] == chosenClass]
consideredImageFilename = consideredImageObs["Image"].iloc[0]

# Observe Class Image

Let's just explore this particular class image.

In [ ]:
fullImageFilename = f'{inputDir}/train/{consideredImageFilename}'
chosenImage = Image.open(fullImageFilename)

In [ ]:
plt.imshow(chosenImage)
#get rid of axes
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)

_Figure 1: Our chosen image for our preprocessing examples._

# Image Resizing

In [ ]:
chosenImage.size

We see that this image is currently $1050 \times 525$ in size. Based on our [previous analysis](https://www.kaggle.com/mmrosenb/whales-an-exploration), the largest image size group is $1050 \times 600$. Given that I prefer having the fewest image resizings as possible, I think $1050 \times 600$ is a good choice for resizing all of the images. Thus, I will resize this to be slightly bigger than it currently is. We will use nearest neighbors to fill in the space.

In [ ]:
idealWidth = 1050
idealHeight = 600
resizedChosenImage = chosenImage.resize((idealWidth,idealHeight),Image.NEAREST)

In [ ]:
plt.imshow(resizedChosenImage)
#get rid of axes
cur_axes = plt.gca()
cur_axes.axes.get_xaxis().set_visible(False)
cur_axes.axes.get_yaxis().set_visible(False)

_Figure 2: The Resized Chosen Image._

Seems like the resizing didn't affect that much, which is good!

# Greyscaling

The first thing we want to do is make an image into a greyscale. We can do this with `PIL`, thankfully.

In [ ]:
greyChosenImage = resizedChosenImage.convert("LA")

In [ ]:
#then plot
fig, axes = plt.subplots(1,2)
fig.set_size_inches(figWidth,figHeight)
axes[0].imshow(resizedChosenImage)
axes[1].imshow(greyChosenImage)

_Figure 3: Grey-Scaling of our Chosen Image._

This allows us to multiply the number of images in a class by at most $2$. Obviously, if a class is already greyscaled, then we can't multiply the number of images in this manner

# Coloring

I had an idea for how to potentially multiply the number of greyscaled images we have by coloring said images.

1. Greyscale all color images.
2. Using a Convolution-to-Convolution network, use the greyscaled images to predict the colored images. This gives us a pipeline between grey images to colored images.
3. Then, map all greyscaled examples through this network to get colored versions of these images.

Given how important the color and patterns of a whale is to the actual identification of it, it might be important to have these example for predictive purposes. That being said, this kind of prediction task is out of the scope of this particular notebook. We might be interested in trying this later down the line.

# Random Rotation

For the next sections, I will be using the parameters suggested by [this notebook](https://www.kaggle.com/lextoumbourou/humpback-whale-id-data-and-aug-exploration).

Let's start with some random rotations.

In [ ]:
#convert image to array
imageArray = np.array(resizedChosenImage)

In [ ]:
numRotations = 4
rotationSize = 30
#make rotation
rotatedImages = [
    kim.random_rotation(imageArray,rotationSize, 
                        row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    for _ in range(numRotations)]

In [ ]:
#then plot each
fig, givenSubplots = plt.subplots(2,2)
fig.set_size_inches(figWidth,figHeight)
for i in range(len(rotatedImages)):
    givenSubplots[int(i / 2),i % 2].imshow(rotatedImages[i])

_Figure 4: Rotating our Image around  $30$ degrees._

We see that this image remains recognizable despite its slight rotation. I would say this is a strong reason to consider rotations for bumping up the image set.

# Random Shift

Let's try shifting the image some small amount.

In [ ]:
numShifts = numRotations
widthRange = 0.1
heightRange = 0.3
shiftedImages = [
    kim.random_shift(imageArray, wrg= widthRange, hrg= heightRange, 
                     row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    for _ in range(numShifts)]

In [ ]:
#then plot each
fig, givenSubplots = plt.subplots(2,2)
fig.set_size_inches(figWidth,figHeight)
for i in range(len(shiftedImages)):
    givenSubplots[int(i / 2),i % 2].imshow(shiftedImages[i])

_Figure 5: Some Image Shifts._

Some of these shifts may be a bit too severe for our work. one thing we may want to do is lower the height shift range just a tad bit to be more in line with the width shift range.

# Random Shear

I have no idea what a shear does, but if someone could provide reference on it, that would be great. All I know is that is does something to the images in transformation. Let's see what it does!

In [ ]:
numShears = numRotations
givenIntensity = 0.4
shearedImages = [
    kim.random_shear(imageArray, intensity= givenIntensity, 
                 row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    for _ in range(numShears)]

In [ ]:
#then plot
fig, givenSubplots = plt.subplots(2,2)
fig.set_size_inches(figWidth,figHeight)
for i in range(len(shearedImages)):
    givenSubplots[int(i / 2),i % 2].imshow(shearedImages[i])

_Figure 6: Some sheared versiions of our image._

Still can't tell with shearing is doing from these images. Anyone have suggestions on this?

# Random Zoom

Given that these whales have flukes that come in all shapes and sizes, let's try to add some randomization via a zooming method.

In [ ]:
numZooms = numRotations
zoomRangeWidth = 1.5
zoomRangeHeight = .7
zoomedImages = [
    kim.random_zoom(imageArray, zoom_range=(zoomRangeWidth,zoomRangeHeight),
                row_axis=0, col_axis=1, channel_axis=2, fill_mode='nearest')
    for _ in range(numZooms)]

In [ ]:
#then plot
fig, givenSubplots = plt.subplots(2,2)
fig.set_size_inches(figWidth,figHeight)
for i in range(len(zoomedImages)):
    givenSubplots[int(i / 2),i % 2].imshow(zoomedImages[i])

_Figure 7: Zoomed versions of our image._

We see that a lot of variation begins to show in this method. In my opinion, this will probably have the strongest impact on generalization, because not all cameras can get close enough to get the best picture of the fluke.

# Redscaling

As suggested by my [previous analysis](https://www.kaggle.com/mmrosenb/whales-an-exploration), there is some evidence of redscaling in the training and test dataset. Thus, we should have our pictures be robust to this scale. While I still haven't figured out a way to identify redscaling in a picture, it is pretty straightforward to transfer a current picture into redscale.

In [ ]:
#collapse to redscale
redImageArray = copy.deepcopy(imageArray)
redImageArray[:,:,1] = 0
redImageArray[:,:,2] = 0

In [ ]:
plt.imshow(redImageArray)

_Figure 8: Redscaling of image._

This is not the kind of redscaling we are looking for. This is much too dark when compared to the redscaling present in [my previous analysis](https://www.kaggle.com/mmrosenb/whales-an-exploration). Thus, I think I'm going to leave redscaling out of the transformation set until I have this right.

# Better Red-Scaling

Let's try redscaling via [these instructions](http://nicolas.riousset.com/category/software-methodologies/basic-color-scale-conversion-algorithm/).

In [ ]:
colorEnhancer = 30
averageArray = np.mean(imageArray,axis = 2)
redImageArray = copy.deepcopy(imageArray)
redImageArray[:,:,1] = averageArray
redImageArray[:,:,2] = averageArray
redImageArray[:,:,0] += colorEnhancer

In [ ]:
plt.imshow(redImageArray)

_Figure 9: New Redscaling of Image._

Looks like that's not the kind of redscaling we are looking for either. We'll need to get back to this to get the redscaling we are looking for as in [this analysis](https://www.kaggle.com/mmrosenb/whales-an-exploration).

# Other transformation options

* One other possible transformation option is the vertical flip, but as suggested by [this notebook](https://www.kaggle.com/lextoumbourou/humpback-whale-id-data-and-aug-exploration), since vertical alignment is crucial between Flukes, it might be better to leave this transformation out of consideration

# Number of images post-transformation

Let's do a quick count of the number of images you can create if you keep applying the five transformations above in a sequential manner.

## Color Picture

This allows us to start with two pictures: the color version and the greyscale version. Say that we apply four randomized versions of each transformation sequentially.

In [ ]:
initialNumImages = 2
numberOfTransformations = 4
numRandomIterationsOfTransformation = 4
numImages = initialNumImages
#then apply sequentially
for i in range(numberOfTransformations):
    numImages += (numImages * numRandomIterationsOfTransformation)
numImages

If we keep applying these, this gets us $1250$ new images! Even bigger than the largest class (see [this analysis](https://www.kaggle.com/mmrosenb/whales-an-exploration)).

# Grayscaled Picture

Since it is already grayscaled, we only start with one image.

In [ ]:
initialNumImages = 1
numberOfTransformations = 4
numRandomIterationsOfTransformation = 4
numImages = initialNumImages
#then apply sequentially
for i in range(numberOfTransformations):
    numImages += (numImages * numRandomIterationsOfTransformation)
numImages

Only half, as expected. Still, this gets us close the largest class of $810$ images (see [this analysis](https://www.kaggle.com/mmrosenb/whales-an-exploration) for details).

# Conclusion

We can significantly increase the size of our small classes by sequentially applying various transformations to our images. Of course, we will have to make sure these transformations do not alter the availability of the fluke in the image, else this would defeat the purpose of making these transformations. Nonetheless, This should be a step in the right direction in balancing our classes.

We still need to figure out how to apply an appropriate redscaling algorithm. Perhaps instructions on the internet [like these](https://stackoverflow.com/questions/5976153/how-to-convert-an-image-to-redscale).